# Proyek Klasifikasi Gambar Capstone: Batiku


## Import Semua Packages/Library yang Digunakan

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


## Data Preparation

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Unzip dataset
dataset_path = "/content/drive/MyDrive/mesin/clasification_dataset/batik.zip"

In [ ]:
# Ekstrak file zip
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

# Pastikan dataset berhasil diekstrak
print("Ekstraksi selesai. File tersedia di /content/dataset")

In [ ]:
# Melihat isi folder dataset
dataset_dir = "/content/dataset"
print("Folder dalam dataset:", os.listdir(dataset_dir))

In [ ]:
total = sum(len(os.listdir(os.path.join(dataset_dir, label))) for label in os.listdir(dataset_dir))
print(f"\nTotal dataset: {total}")

### Data Preprocessing

#### Split Dataset

In [ ]:
train_dir = "/content/dataset_split/train"
val_dir = "/content/dataset_split/val"
test_dir = "/content/dataset_split/test"

from sklearn.model_selection import train_test_split
import shutil

def split_dataset(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.7, 0.15, 0.15)):
    labels = os.listdir(source_dir)
    for label in labels:
        img_paths = [os.path.join(source_dir, label, fname) for fname in os.listdir(os.path.join(source_dir, label))]
        train_files, temp_files = train_test_split(img_paths, train_size=split_ratio[0], shuffle=True)
        val_files, test_files = train_test_split(temp_files, test_size=split_ratio[2]/(split_ratio[1]+split_ratio[2]))

        for path_set, target_dir in zip([train_files, val_files, test_files], [train_dir, val_dir, test_dir]):
            label_dir = os.path.join(target_dir, label)
            os.makedirs(label_dir, exist_ok=True)
            for file in path_set:
                shutil.copy(file, label_dir)

# Jalankan fungsi split
split_dataset("/content/dataset", train_dir, val_dir, test_dir)


# Preprocessing dan Augmentasi Gambar

In [ ]:
img_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_test_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)


## Modelling

# CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# LATIH MODEL

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)

# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

## Evaluasi dan Visualisasi

In [ ]:
# Plot akurasi dan loss selama training
plt.figure(figsize=(12, 4))

# Akurasi
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Graph')

plt.show()

# CONVOLUTION MATRIX

In [ ]:
# Confusion Matrix & Classification Report
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
labels = list(train_generator.class_indices.keys())

print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=labels))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

## Konversi Model

In [ ]:
pip install tensorflowjs==4.12.0

In [ ]:
import os
import tensorflowjs as tfjs

# Membuat folder tujuan di Google Drive
os.makedirs("/content/drive/MyDrive/mesin/tfjs_model", exist_ok=True)
os.makedirs("/content/drive/MyDrive/mesin/tflite", exist_ok=True)
os.makedirs("/content/drive/MyDrive/mesin/saved_model", exist_ok=True)

# 1. Simpan model untuk TensorFlow.js
tfjs.converters.save_keras_model(model, "/content/drive/MyDrive/mesin/tfjs_model")

# 2. Simpan model untuk TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("/content/drive/MyDrive/mesin/tflite/model.tflite", "wb") as f:
    f.write(tflite_model)

# Simpan label
labels = list(train_generator.class_indices.keys())  # pastikan daftar label tersedia
with open("/content/drive/MyDrive/mesin/tflite/label.txt", "w") as f:
    f.write("\n".join(labels))

In [ ]:
model.export("/content/drive/MyDrive/mesin/saved_model")

## Inference

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image



# Class labels (should match your training data)
class_labels = []

def preprocess_image(img_path, target_size=(150, 150)):
    """Load and preprocess an image for prediction"""
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize like during training
    return img_array

def predict_disease(img_path, model, class_labels):
    """Make a prediction on a single image"""
    # Preprocess the image
    processed_img = preprocess_image(img_path)

    # Make prediction
    predictions = model.predict(processed_img)
    predicted_class_index = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    predicted_class = class_labels[predicted_class_index]

    # Display results
    img = image.load_img(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2%}")
    plt.show()

    # Print detailed probabilities
    print("\nPrediction probabilities:")
    for i, prob in enumerate(predictions[0]):
        print(f"{class_labels[i]}: {prob:.4f}")

    return predicted_class, confidence

image_path = ""
predicted_class, confidence = predict_disease(image_path, model, class_labels)